In [1]:
from datasets import load_dataset


In [2]:
import pandas as pd
import numpy as np
import re
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer

In [3]:
from sentence_transformers import SentenceTransformer, util
import torch

In [4]:
from scipy.spatial.distance import cdist
from sklearn.metrics import ndcg_score

In [5]:
dataset_ruby = load_dataset("code_search_net", "ruby")
dataset_go = load_dataset("code_search_net", "go")

Found cached dataset code_search_net (/Users/christinexu/.cache/huggingface/datasets/code_search_net/ruby/1.0.0/8f2524e6b62f65af5f5d65c53715c654db7b08dc93e0b7bcce2ab2f286a75be1)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset code_search_net (/Users/christinexu/.cache/huggingface/datasets/code_search_net/go/1.0.0/8f2524e6b62f65af5f5d65c53715c654db7b08dc93e0b7bcce2ab2f286a75be1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# type(dataset)

NameError: name 'dataset' is not defined

In [7]:
ground_truth = pd.read_csv('annotationStore.csv')
gt_ruby = ground_truth[ground_truth['Language'] == 'Ruby']
gt_go = ground_truth[ground_truth['Language'] == 'Go']

In [8]:
train_go = pd.DataFrame(dataset_go['train'])
train_go.head()

,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,func_code_url
0,aws/aws-sdk-go,service/alexaforbusiness/api.go,Validate,func (s *UpdateSkillGroupInput) Validate() err...,go,func (s *UpdateSkillGroupInput) Validate() err...,"[func, (, s, *, UpdateSkillGroupInput, ), Vali...",// Validate inspects the fields of the type to...,"[Validate, inspects, the, fields, of, the, typ...",train,https://github.com/aws/aws-sdk-go/blob/6c40606...
1,aws/aws-sdk-go,service/alexaforbusiness/api.go,SetDescription,func (s *UpdateSkillGroupInput) SetDescription...,go,func (s *UpdateSkillGroupInput) SetDescription...,"[func, (, s, *, UpdateSkillGroupInput, ), SetD...",// SetDescription sets the Description field's...,"[SetDescription, sets, the, Description, field...",train,https://github.com/aws/aws-sdk-go/blob/6c40606...
2,aws/aws-sdk-go,service/alexaforbusiness/api.go,SetSkillGroupArn,func (s *UpdateSkillGroupInput) SetSkillGroupA...,go,func (s *UpdateSkillGroupInput) SetSkillGroupA...,"[func, (, s, *, UpdateSkillGroupInput, ), SetS...",// SetSkillGroupArn sets the SkillGroupArn fie...,"[SetSkillGroupArn, sets, the, SkillGroupArn, f...",train,https://github.com/aws/aws-sdk-go/blob/6c40606...
3,aws/aws-sdk-go,service/alexaforbusiness/api.go,SetSkillGroupName,func (s *UpdateSkillGroupInput) SetSkillGroupN...,go,func (s *UpdateSkillGroupInput) SetSkillGroupN...,"[func, (, s, *, UpdateSkillGroupInput, ), SetS...",// SetSkillGroupName sets the SkillGroupName f...,"[SetSkillGroupName, sets, the, SkillGroupName,...",train,https://github.com/aws/aws-sdk-go/blob/6c40606...
4,aws/aws-sdk-go,service/alexaforbusiness/api.go,SetEmail,func (s *UserData) SetEmail(v string) *UserDat...,go,func (s *UserData) SetEmail(v string) *UserDat...,"[func, (, s, *, UserData, ), SetEmail, (, v, s...",// SetEmail sets the Email field's value.,"[SetEmail, sets, the, Email, field, s, value, .]",train,https://github.com/aws/aws-sdk-go/blob/6c40606...


In [9]:
train_ruby = pd.DataFrame(dataset_ruby['train'])

In [10]:
merged_ruby = gt_ruby.merge(train_ruby, left_on = 'GitHubUrl', right_on = 'func_code_url')
merged_ruby.shape

(109, 16)

In [11]:
merged_go = gt_go.merge(train_go, left_on = 'GitHubUrl', right_on = 'func_code_url')
merged_go.shape

(76, 16)

# EDA of ground truth and Ruby/Go dataset

In [12]:
gt_ruby[gt_ruby['Query'] == 'write csv']['GitHubUrl'].loc[4001]

'https://github.com/khiemns54/sp2db/blob/76c78df07ea19d6f1b5ff2e883ae206a0e94de27/lib/sp2db/base_table.rb#L105-L110'

In [13]:
train_df.head()

NameError: name 'train_df' is not defined

In [14]:
target = 'https://github.com/angelim/rails_redshift_replicator/blob/823f6da36f43a39f340a3ca7eb159df58a86766d/lib/rails_redshift_replicator/file_manager.rb#L49-L51'

In [15]:
target in list(gt_ruby['GitHubUrl'])

False

In [16]:
# merged = pd.concat([merged_ruby, merged_go], ignore_index=True)
merged = merged_ruby

## Directly throw everything in
https://huggingface.co/flax-sentence-embeddings/st-codesearch-distilroberta-base

In [17]:
# model_code = SentenceTransformer("flax-sentence-embeddings/st-codesearch-distilroberta-base")
model_code = SentenceTransformer("all-MiniLM-L6-v2")
model_doc = SentenceTransformer('all-MiniLM-L6-v2')
model_idt = SentenceTransformer('all-MiniLM-L6-v2')

In [18]:
# sample_code = dataset['train'][:500]['whole_func_string']
# sample_idt = dataset['train'][:500]['func_name']
# sample_doc = dataset['train'][:500]['func_documentation_string']
sample_code = merged['whole_func_string']
sample_idt = merged['func_name']
sample_doc = merged['func_documentation_string']

In [19]:
# are there labeled dataset and sample queries?
# print(dataset['train'][20]['func_code_string'])

In [20]:
# idx = 0


In [21]:
# # (language, identifier, code)
# (dataset['train'][idx]['language'], dataset['train'][idx]['func_name'] + dataset['train'][idx]['func_documentation_string'], dataset['train'][idx]['func_code_string'])


In [22]:
# class SemanticSearch:
    
#     def __init__(self, )

In [23]:
# def search(query, language):
#     # if we want to match separately on fields(i.e. code and documentation/identifier)
#     score_code = 0.0
#     score_doc = 0.0
#     score_idt = 0.0
    
# #     if language == "python":
# #         ...
    
#     code_emb = model.encode(sample_train, convert_to_tensor=True)
#     doc_emb = model.encode(sample_doc, convert_to_tensor=True)
#     idt_emb = model.encode(sample_idt, convert_to_tensor=True)
    

In [24]:
code_emb = model_code.encode(sample_code, convert_to_tensor=True)
doc_emb = model_doc.encode(sample_doc, convert_to_tensor=True)
idt_emb = model_idt.encode(sample_idt, convert_to_tensor=True)

In [25]:
query = 'get name of enumerated value'

### doc embedding

In [43]:
que_doc_emb = model_doc.encode(query, convert_to_tensor=True)
hits = util.semantic_search(que_doc_emb, doc_emb)[0]
top_hits = hits[:3]
# cdist_score = cdist(que_doc_emb, doc_emb)
# top_hits = torch.topk(cdist_score, k=top_3)
rank = 1
que_doc_df = pd.DataFrame()
que_doc_score = []
que_doc_url = []
que_doc_rank = []
for top_hit in top_hits:
    print("Cossim: {:.2f}".format(top_hit['score']))
    print(f"Rank: {rank}")
    print(merged.iloc[top_hit['corpus_id']]['func_code_url'])
    print("\n\n")
    
    que_doc_score.append(top_hit['score'])
    que_doc_url.append(merged.iloc[top_hit['corpus_id']]['func_code_url'])
    que_doc_rank.append(rank)
    rank += 1

que_doc_df['score'] = que_doc_score
que_doc_df['rank'] = que_doc_rank
que_doc_df['url'] = que_doc_url
que_doc_df

Cossim: 0.33
Rank: 1
https://github.com/garethlatwork/quickbase_client/blob/13d754de1f22a466806c5a0da74b0ded26c23f05/lib/QuickBaseClient.rb#L914-L938



Cossim: 0.29
Rank: 2
https://github.com/dpla/KriKri/blob/0ac26e60ce1bba60caa40263a562796267cf833f/lib/krikri/harvesters/oai_harvester.rb#L159-L165



Cossim: 0.28
Rank: 3
https://github.com/booqable/scoped_serializer/blob/fb163bbf61f54a5e8684e4aba3908592bdd986ac/lib/scoped_serializer/array_serializer.rb#L29-L39





,score,rank,url
0,0.327444,1,https://github.com/garethlatwork/quickbase_cli...
1,0.285970,2,https://github.com/dpla/KriKri/blob/0ac26e60ce...
2,0.279068,3,https://github.com/booqable/scoped_serializer/...


### code embedding

In [29]:
que_code_emb = model_code.encode(query, convert_to_tensor=True)
hits = util.semantic_search(que_code_emb, code_emb)[0]
top_hits = hits[:3]

for top_hit in top_hits:
    print("Cossim: {:.2f}".format(top_hit['score']))
    print(merged.iloc[top_hit['corpus_id']]['func_code_url'])
    print("\n\n")

Cossim: 0.49
https://github.com/dpla/KriKri/blob/0ac26e60ce1bba60caa40263a562796267cf833f/lib/krikri/harvesters/oai_harvester.rb#L159-L165



Cossim: 0.28
https://github.com/opal/opal/blob/41aedc0fd62aab00d3c117ba0caf00206bedd981/opal/corelib/marshal/read_buffer.rb#L388-L407



Cossim: 0.27
https://github.com/robertwahler/win32-autogui/blob/f36dc7ed9d7389893b0960b2c5740338f0d38117/lib/win32/autogui/window.rb#L218-L223





### identifier embedding

In [30]:
que_idt_emb = model_idt.encode(query, convert_to_tensor=True)
hits = util.semantic_search(que_idt_emb, idt_emb)[0]
top_hits = hits[:3]
for top_hit in top_hits:
    print("Cossim: {:.2f}".format(top_hit['score']))
    print(merged.iloc[top_hit['corpus_id']]['func_code_url'])
    print("\n\n")

Cossim: 0.32
https://github.com/mdub/lazily/blob/8665e3532ea2d2ea3a41e6ec6a47088308979754/lib/lazily/filtering.rb#L63-L75



Cossim: 0.29
https://github.com/dpla/KriKri/blob/0ac26e60ce1bba60caa40263a562796267cf833f/lib/krikri/harvesters/oai_harvester.rb#L159-L165



Cossim: 0.23
https://github.com/mare-imbrium/canis/blob/8bce60ff9dad321e299a6124620deb4771740b0b/lib/canis/core/widgets/deprecated/rtextview.rb#L193-L198





### Combined

In [ ]:
# util.semantic_search(que_code_emb, code_emb)[0]['']

In [ ]:
# hits = 6 * util.semantic_search(que_code_emb, code_emb)[0] + 1* util.semantic_search(que_idt_emb, idt_emb)[0] + 3* util.semantic_search(que_doc_emb, doc_emb)[0]
# top_hits = hits[:3]
# for top_hit in top_hits:
#     print("Cossim: {:.2f}".format(top_hit['score']))
#     print(sample_train[top_hit['corpus_id']])
#     print("\n\n")

In [ ]:
# dataset['test'][0].keys()

In [ ]:
# code_emb = model.encode(sample_train, convert_to_tensor=True)

In [ ]:
# query = 'depth first search'

In [ ]:
# query_emb = model.encode(query, convert_to_tensor=True)
# hits = util.semantic_search(query_emb, code_emb)[0]
# top_hits = hits[:3]

# for top_hit in top_hits:
#     print("Cossim: {:.2f}".format(top_hit['score']))
#     print(sample_train[top_hit['corpus_id']])
#     print("\n\n")

## Evaluation (nDCG)
- how to decide the "correct" rank? Do we have to run the whole train dataset on DSMLP and then follow the examples?
- Relevance score? still use recipricol rank


In [ ]:
3,1,3
true_relevance = np.asarray([[0.3,1,0.3, 0]])
# we predict some scores (relevance) for the answers
scores = np.asarray([[1,0.5,0.3, 0.25]])
ndcg_score(true_relevance, scores)

In [49]:
target = merged[merged['Query'] == query]
target

,Language,Query,GitHubUrl,Relevance,Notes,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,func_code_url
74,Ruby,get name of enumerated value,https://github.com/dpla/KriKri/blob/0ac26e60ce...,1,NaN,dpla/KriKri,lib/krikri/harvesters/oai_harvester.rb,Krikri::Harvesters.OAIHarvester.concat_enum,def concat_enum(enum_enum)\n Enumerator.n...,ruby,def concat_enum(enum_enum)\n Enumerator.n...,"[def, concat_enum, (, enum_enum, ), Enumerator...",Concatinates two enumerators\n @todo find a be...,"[Concatinates, two, enumerators]",train,https://github.com/dpla/KriKri/blob/0ac26e60ce...


In [50]:
# gt_ruby[gt_ruby['Query'] == query]

In [45]:
final = target.merge(que_doc_df, left_on = 'GitHubUrl', right_on = 'url')

In [46]:
final

,Language,Query,GitHubUrl,Relevance,Notes,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,func_code_url,score,rank,url
0,Ruby,get name of enumerated value,https://github.com/dpla/KriKri/blob/0ac26e60ce...,1,NaN,dpla/KriKri,lib/krikri/harvesters/oai_harvester.rb,Krikri::Harvesters.OAIHarvester.concat_enum,def concat_enum(enum_enum)\n Enumerator.n...,ruby,def concat_enum(enum_enum)\n Enumerator.n...,"[def, concat_enum, (, enum_enum, ), Enumerator...",Concatinates two enumerators\n @todo find a be...,"[Concatinates, two, enumerators]",train,https://github.com/dpla/KriKri/blob/0ac26e60ce...,0.28597,2,https://github.com/dpla/KriKri/blob/0ac26e60ce...


In [47]:
MRR = 1/final['rank']

In [48]:
MRR

0    0.5
Name: rank, dtype: float64